<a href="https://colab.research.google.com/github/EN10/BabyLlama/blob/main/Baby_Llama_105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# training and inference code
!git clone https://github.com/karpathy/llama2.c.git
# tokenizer library
#!pip install sentencepiece

Cloning into 'llama2.c'...
remote: Enumerating objects: 1507, done.
remote: Counting objects: 100% (640/640), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 1507 (delta 601), reused 580 (delta 566), pack-reused 867
Receiving objects: 100% (1507/1507), 1.20 MiB | 3.85 MiB/s, done.
Resolving deltas: 100% (926/926), done.


In [2]:
# download training data
!cd llama2.c && python tinystories.py download

data/TinyStories_all_data.tar.gz: 100% 1.50G/1.50G [00:33<00:00, 48.4MiB/s]
Unpacking data/TinyStories_all_data.tar.gz...
Download done.
Number of shards: 50
Example story:
{'story': '\n\nLily and Ben are friends. They like to play in the park. One day, they see a big tree with a swing. Lily wants to try the swing. She runs to the tree and climbs on the swing.\n"Push me, Ben!" she says. Ben pushes her gently. Lily feels happy. She swings higher and higher. She laughs and shouts.\nBen watches Lily. He thinks she is cute. He wants to swing too. He waits for Lily to stop. But Lily does not stop. She swings faster and faster. She is having too much fun.\n"Can I swing too, Lily?" Ben asks. Lily does not hear him. She is too busy swinging. Ben feels sad. He walks away.\nLily swings so high that she loses her grip. She falls off the swing. She lands on the ground. She hurts her foot. She cries.\n"Ow, ow, ow!" she says. She looks for Ben. She wants him to help her. But Ben is not there. He is 

In [3]:
# remove byte fallback <0x00> ... <0xFF> on line 113
!sed -i "113s/.*/                                   byte_fallback=False,/" ./llama2.c/tinystories.py
# change bfloat16 to float16 on line 72
!sed -i "72s/.*/dtype = 'float16'  # float32|bfloat16|float16/" ./llama2.c/train.py

In [4]:
# create vocab
#!cd llama2.c && python tinystories.py train_vocab --vocab_size=105
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok105/tok105.model
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/raw/main/tok105/tok105.vocab

# use vocab to pretokenize training data
#!cd llama2.c && python tinystories.py pretokenize --vocab_size=105
#!cd llama2.c/data && tar -czvf tok105.tar.gz tok105
!cd ./llama2.c/data && wget https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok105/tok105.tar.gz
!cd ./llama2.c/data && tar -xf tok105.tar.gz

--2024-02-29 08:14:07--  https://huggingface.co/datasets/enio/TinyStories/resolve/main/tok105/tok105.model
Resolving huggingface.co (huggingface.co)... 18.155.173.64, 18.155.173.122, 18.155.173.126, ...
Connecting to huggingface.co (huggingface.co)|18.155.173.64|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/65/c8/65c8615bfd6b6d94c8e62381ee6653578b439f71ff04b86aaefa9828dec8bb70/eed6827c1b4d876b982b9b628626dba444960a041857085e0d88c4a92aa8d3c6?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27tok105.model%3B+filename%3D%22tok105.model%22%3B&Expires=1709453647&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTQ1MzY0N319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzY1L2M4LzY1Yzg2MTViZmQ2YjZkOTRjOGU2MjM4MWVlNjY1MzU3OGI0MzlmNzFmZjA0Yjg2YWFlZmE5ODI4ZGVjOGJiNzAvZWVkNjgyN2MxYjRkODc2Yjk4MmI5YjYyODYyNmRiYTQ0NDk2MGEwNDE4NTcwODVlMGQ4OGM0YTk

In [5]:
# train neural net
!cd llama2.c && python train.py --vocab_source=custom --vocab_size=105 --compile=False \
  --dim=128 --n_layers=5 --n_heads=8 --n_kv_heads=4 --batch_size=32 \
  --always_save_checkpoint=True --eval_interval=1 --max_iters=1 #--init_from='resume'

Overriding: vocab_source = custom
Overriding: vocab_size = 105
Overriding: compile = False
Overriding: dim = 128
Overriding: n_layers = 5
Overriding: n_heads = 8
Overriding: n_kv_heads = 4
Overriding: batch_size = 32
Overriding: always_save_checkpoint = True
Overriding: eval_interval = 1
Overriding: max_iters = 1
tokens per iteration will be: 32,768
breaks down as: 4 grad accum steps * 1 processes * 32 batch size * 256 max seq len
Initializing a new model from scratch
num decayed parameter tensors: 36, with 935,040 parameters
num non-decayed parameter tensors: 11, with 1,408 parameters
using fused AdamW: True
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 0: train loss 4.7026, val loss 4.7022
0 | loss 4.7046 | lr 0.000000e+00 | 5463.24ms | mfu -100.00%
Created a PretokDataset with rng seed 42
Created a PretokDataset with rng seed 42
step 1: train loss 4.7026, val loss 4.7022
saving checkpoint to out
wrote 

In [6]:
# compile run / inference executable
!cd llama2.c && gcc -O3 -o run run.c -lm
# create tok105.bin
!cd llama2.c && python tokenizer.py --tokenizer-model=data/tok105.model

In [7]:
# inference
!cd ./llama2.c && ./run out/model.bin -z data/tok105.bin -t 0.8 -n 256 -i "Once upon a time "

Once upon a time  b-lxOLu’~f‘8T<unk>Puhgr”D<unk><unk>‘“:ipoa25*OS9|v7&ggC““9]TMXffw  C€/“”™I&x"0Uz€–QQT9znO’6  —q™™|'Lx&”HHH&r:yy76k——x<unk>V|$mOX‘‘<unk>O(”6€G~"
</s>
–cceAe44$2-~  9F**(MM<unk>‘‘QWW RoqqU  $9:roo
</s>
TJFs$ñ™XFs&4HGGXXXXOYñs>eII>)99y;\’2-:&4~lM,ZZ6~
</s>
GéUUDUUIOOW$R7
achieved tok/s: 686.980609
